# Feature_Engineering

In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("train.csv")

'property_type'  sutununu 5 gruba ayirmis

In [ ]:
dict1 = {'Apartment':['Condominium','Timeshare','Loft','Serviced apartment','Guest suite'],
         'House':['Vacation home','Villa','Townhouse','In-law','Casa particular'],
         'Hotel1':['Dorm','Hostel','Guesthouse'],
         'Hotel2':['Boutique hotel','Bed & Breakfast'],
         'Other':['Island','Castle','Yurt','Hut','Chalet','Treehouse',
                  'Earth House','Tipi','Cave','Train','Parking Space','Lighthouse',
                 'Tent','Boat','Cabin','Camper/RV','Bungalow']
        }
dict2 = {i : k for k, v in dict1.items() for i in v}
data['property_type'].replace(dict2).value_counts()

mahalle sayisini 619 bulmus,city ye göre gruplamis ve mahalleyi drop etme karari almis

city  neighbourhood	
Boston	34
Chicago	80
DC	    116
LA	    157
NYC	   201
SF	   55

In [ ]:
data['neighbourhood'].nunique()

pd.DataFrame(data.groupby(['city'])['neighbourhood'].nunique())

one hat encoding yapacagi sutunlari bu sekilde secmis

In [ ]:
#columns for get_dummies (one hot encoding)
col_to_getdummies=['property_type','room_type','bed_type','cancellation_policy','city']

#columns that won't be changed
col_no_change=['accommodates','number_of_reviews']